In [ ]:
import pulp

def optimize_battery_charging(prices, capacity, charging_efficiency, discharging_efficiency, losses, charging_power, discharging_power):
    num_periods = len(prices)

    # Create the LP problem instance
    prob = pulp.LpProblem("ChargeBattery", pulp.LpMaximize)

    # Define decision variables
    x = pulp.LpVariable.dicts("Charge", range(num_periods), lowBound=0, upBound=charging_power)
    y = pulp.LpVariable.dicts("Discharge", range(num_periods), lowBound=0, upBound=discharging_power)
    B = pulp.LpVariable.dicts("SOC", range(num_periods+1), lowBound=0, upBound=capacity)

    # Set the objective function
    prob = pulp.lpSum(prices[t] * y[t] - prices[t] * x[t] - losses(x[t], y[t]) for t in range(num_periods))
    #HERE!!!!!!!!!!!!!!!!!!!!!!!OOOOOOOOOOOOOOOOOOOOOOOOÖÖÖÖÖÖÖÖWÖSÖSÖSÖSÖSÖSÖSÖSÖSÖSÖSÖSÖSÖ
    #prob += pulp.lpSum(prices[t] * y[t] - prices[t] * x[t] - losses(x[t], y[t]) for t in range(num_periods))

    # Set constraints
    for t in range(num_periods):
        # Battery state update
        prob += B[t+1] == B[t] + charging_efficiency * x[t] - y[t] / discharging_efficiency

        # Battery capacity constraints
        prob += B[t+1] <= capacity

        # Energy balance constraint
        prob += x[t] == 0 if B[t] == capacity else None
        prob += y[t] == 0 if B[t] == 0 else None

    # Solve the problem
    prob.solve()

    # Retrieve the optimal solution
    optimal_x = [value(x[t]) for t in range(num_periods)]
    optimal_y = [value(y[t]) for t in range(num_periods)]

    return optimal_x, optimal_y

# Example usage
prices = [0.10, 0.15, 0.12, 0.18, 0.14]  # Electricity prices for each time step
capacity = 100  # Battery capacity
charging_efficiency = 0.9  # Charging efficiency (e.g., 90%)
discharging_efficiency = 0.85  # Discharging efficiency (e.g., 85%)

# Define the losses function based on your specific charging and discharging losses calculation
def losses(x, y):
    # Example calculation: assume 2% losses per unit of electricity transferred
    return 0.02 * (x + y)

optimal_buy, optimal_sell = optimize_buy_sell_electricity(prices, capacity, charging_efficiency, discharging_efficiency, losses)

print("Optimal Buy: ", optimal_buy)
print("Optimal Sell: ", optimal_sell)
